<a href="https://colab.research.google.com/github/LarkinConor/Resume/blob/main/Conor_LarkinA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("/content/sample_data/River-and-Rail-Cantina.csv")

In [ ]:
df['user_name'] = df['user_name'].astype(str)
df['rating'] = df['rating'].astype(str)
df['rating'] = df['rating'].str.replace(' star rating', '')
df['review_text'] = df['review_text'].astype(str)

In [ ]:
df['user_name'] = df['user_name'].str.strip("['']")
df['rating'] = df['rating'].str.strip("['']")
df['review_text'] = df['review_text'].str.strip("['']")

In [ ]:
df['rating'] = df['rating'].str.split("'", expand=True).get(0) #Some people reviewed the restaurant twice and I couldn't figure out how to split the reviews so I just deleted the second review
df['user_name'] = df['user_name'].str.split("'", expand=True).get(0) #Some people had the amount of photos posted so I cleaned it up
df['review_text'] = df['review_text'].str.split(", '", expand=True).get(0) #same as the ratings problem

In [ ]:
df['rating'] = df['rating'].astype(int)

In [ ]:
df.drop("Unnamed: 0", axis=1, inplace=True) #I do not know where unnamed:  0 came from so I dropped it

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 766 entries, 0 to 765
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   review_date  766 non-null    object
 1   user_name    766 non-null    object
 2   rating       766 non-null    int64 
 3   review_text  766 non-null    object
 4   sentiment    766 non-null    object
dtypes: int64(1), object(4)
memory usage: 30.0+ KB


In [ ]:
def recode_rating(rating):
    if rating <= 3:
        return 'Negative'
    elif rating <= 5:
        return 'Positive'
    else:
        return 'Unknown'

df['sentiment'] = df['rating'].apply(recode_rating)

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [ ]:
def preprocess_review(review_text):
    # Remove punctuation
    review_text = review_text.translate(str.maketrans('', '', string.punctuation))
    
    # Convert all words to lower case
    review_text = review_text.lower()
    
    # Tokenize the review into words
    words = word_tokenize(review_text)
    
    # Remove stop words
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]
    
    # Join the remaining words back into a single string
    processed_review = ' '.join(words)
    
    return processed_review

df['review_text'] = df['review_text'].apply(preprocess_review)

In [ ]:
tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
stop_words = set(stopwords.words('english'))

In [ ]:
normalized_reviews = []
for review in df['review_text']:
    tokens = tokenizer.tokenize(review.lower())
    filtered_tokens = [token for token in tokens if token not in stop_words]
    normalized_reviews.append(filtered_tokens)

In [ ]:
vocabulary = set()
for review in normalized_reviews:
    vocabulary.update(review)

In [ ]:
feature_sets = []
for i in range(len(normalized_reviews)):
    features = {}
    for word in vocabulary:
        features[word] = normalized_reviews[i].count(word)
    label = df['rating'][i]
    feature_sets.append((features, label))

In [ ]:
import random
random.shuffle(feature_sets)

In [ ]:
cutoff = int(0.7 * len(feature_sets))
train_set = feature_sets[:cutoff]
test_set = feature_sets[cutoff:]

In [ ]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

# Evaluate the classifier on the testing set
accuracy = nltk.classify.accuracy(classifier, test_set)
print("Classifier accuracy:", accuracy)

Classifier accuracy: 0.4782608695652174


In [ ]:
classifier.show_most_informative_features(5)

Most Informative Features
                   great = 2                   5 : 1      =     13.6 : 1.0
               excellent = 1                   5 : 1      =     12.3 : 1.0
              definitely = 1                   5 : 2      =     10.8 : 1.0
                   could = 1                   1 : 5      =     10.4 : 1.0
               delicious = 1                   5 : 1      =     10.2 : 1.0


I struggled a lot with this a couldn't random the words of the reviews no matter what I tried so I went to Chatgpt and went from there. I don't exactly know where it went wrong but I think I did get the top five words but it says = 1 when I thought it should be = True



